# Goals of this Script:

The goals of this script are to get an introduction to analysis of the covid 19 data in Europe. I want to see differences and changes over time in the number of confirmed cases, the number of active cases, the number of deaths, and the number of recoveries. Another thing I found was the total number of cases by country and compare them to see which has the most. Lastly, I found the percent makeup of confirmed cases, specifically the percentage of cases that were active, resulted in recoveries, and resulted in death. This is only a start and I always welcome suggestions. I am new to interactive visualizations and am learning, so please feel free to leave suggestions and feedback.



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px



In [ ]:
eur_df_new = pd.read_csv('../input/covid19-cases-in-africa/covid19_europe.csv')
#import europe Cases

In [ ]:
eur_df_new.head()

In [ ]:
eur_df_new.shape
#32138 records, 7 columns. Now we can dive more into the columns and their contents.

In [ ]:
eur_df_new.dtypes
#data types

In [ ]:
pd.unique(eur_df_new['Country_Region'])
#this dataset looks at europe

In [ ]:
eur_df_new.isnull().sum()
#Missing Value Count.
#7515 states or provinces within a country missing here., 24 active cases missing.


In [ ]:
eur_df_new['Province_State'].isnull().sum()/25682
#This depicts the percentage of the Province_States values that are missing. 
#The threshold I go by is that if upwards of 25-30% of the values are missing I drop the column.

In [ ]:
eur_df_new = eur_df_new.drop(columns = 'Province_State')

In [ ]:
eur_df_new.isnull().values.any()

In [ ]:
eur_df_new = eur_df_new.dropna()

In [ ]:
eur_df_new.shape #new shape

In [ ]:
import datetime as dt
#use it to obtain month and year in column for potential grouping purposes

In [ ]:

eur_df_new['ObservationDate'] = pd.to_datetime(eur_df_new['ObservationDate'])
eur_df_new['mnth_yr'] = eur_df_new['ObservationDate'].apply(lambda x: x.strftime('%m-%Y')) 

#change datetime format


In [ ]:
eur_df_new.dtypes
#new data types, the datetime conversion was successful

In [ ]:
eur_df_new.head()


In [ ]:
eur_df_new = eur_df_new.sort_values(by = 'mnth_yr', ascending=True)



In [ ]:
eur_df_new
#new column entry successful

In [ ]:
eur_df_new[['new_confirmed','new_active','new_deaths','new_recoveries']] = (eur_df_new.sort_values
                                                                            (by=['ObservationDate'], ascending=True)
                        .groupby(['Country_Region'])[['Confirmed','Active','Recovered','Deaths']].shift(1))


#eur_df_new['new_actives'] = eur_df_new['Active'] - eur_df_orig['Active']
#eur_df_new['new_recoveries'] = eur_df_new['Recovered'] - eur_df_orig['Recovered']
#eur_df_new['new_deaths'] = eur_df_new['Deaths']- eur_df_orig['Deaths']

In [ ]:
eur_df_new.head(20)

In [ ]:
eur_df_new.dtypes
#new types

# Graphics

The three visuals below will tell us more about the data:

1. An interactive line graph(time series)
2. An interactive pie chart
3. An interactive heatmap


## Trends in Cases over time

We can see that the total number of confirmed cases has increased from January 2020 to August 2020 as well as the deaths and recoveries. However, with the active cases, we see a sharp increase from March to April, and then May to June the rateof increase starts to slow, and then once August hit there was another spike of cases in Europe. With that being said, the reason we saw that is because the amount of recoveries from May to August increased significantly, so we are seeing good progress in Europe on the recovery rate. However, the confirmed cases increasing means Europe definitely needs to be more careful for a bit.

In [ ]:

cases_over_time = eur_df_new.groupby(['ObservationDate'])[['Confirmed','Active','Recovered','Deaths']].agg(np.sum)  
#cases over time by ObservationDate

In [ ]:
cases_over_time.shape 
#total confirmed, active, recovered, deaths, now we can compare across

In [ ]:

cases_over_time
#A table with cases Over time

In [ ]:
cases_over_time.dtypes

In [ ]:
#from numpy import nan
#cases_over_time = cases_over_time.replace(nan,0)
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#matplotlib dateformatter
import matplotlib.figure as fig
import datetime as dt

In [ ]:
cases_over_time.plot()
plt.title('Cases over Time')
plt.yscale('linear')
plt.xlabel('Dates')
plt.ylabel('Total Cases')
#the number of confirmed cases and recovered cases in europe have increased. 
#non interactive plot to look at our dataframe and see the trends of case changes over time.


In [ ]:

cases_ot = eur_df_new.groupby('ObservationDate', as_index=False)[['Confirmed','Active','Recovered','Deaths']].agg(np.sum)


In [ ]:

cases_ot.head()


In [ ]:
cases_ot_long = pd.melt(cases_ot, id_vars = ['ObservationDate'], 
                        value_vars = ['Confirmed','Active','Recovered','Deaths'])
#need to change the data frame shape to use in plotly


In [ ]:
cases_ot_long #new dataframe

In [ ]:
import plotly.graph_objects as go
#import both plotly graph objects now as well as plotly express prior

In [ ]:
cases_ot_long = cases_ot_long.rename(columns = {'ObservationDate':'Date', 'variable':'Case Action Recorded', 
                                                'value':'Total Cases'})
#renaming columns for plot

In [ ]:
figure = px.line(cases_ot_long, x ='Date', y = 'Total Cases', color = 'Case Action Recorded', 
                 title = 'Case Changes Over Time')
figure.show()
#plotly express allows for interactive plots, now we can see the increase by day

In [ ]:
cases_ot_no_confirmed = cases_ot_long[(cases_ot_long['Case Action Recorded']!='Confirmed')]
#only want to look at the makeup of confirmed cases: ex: the total % of confirmed cases that were recovered

# Percent Distribution of Total Confirmed Cases Via Pie Chart

By looking at the chart we see below, we can see the percentage of confirmed cases that are active, recovered, or have resulted in death. 50.1% of the cases in europe have been recovered, 6.69% of those confirmed cases resulted in deaths, and 43.1% of those cases are still active as of today. We can conclude that the number of deaths is low relative to the total number of recoveries, but ideally the number of deaths would be lower. 

In [ ]:
px.pie(cases_ot_no_confirmed, values='Total Cases',names='Case Action Recorded',
       title='Confirmed Case Percentage Makeup')

#percentage of confirmed cases that are active, recovered, and have resulted in deaths.



## Heatmap to show total Cases by Country

The goal of this map is to get an idea of which countries have an outlier in their total number of confirmed cases, active cases, cases that resulted in recovery, and cases that resulted in death. If we see more of that it will tell us which countries are doing the best during this time period covid wise and which are doing the worst.According to our data, Russia has the most recoveries upwards of 85 million cases, while Germany is coming in at a close second. Russia also has the largest total number of confirmed cases, but England has come in a close second and their recovery numbers in the lower echelon at 261.704k, which is abyssmal considering the number of cases. They also have 5 million deaths as well which is terrible considering the amount of recoveries they have had. England has got to do better and that is evident based on the data.

In [ ]:
total_overall_by_country = eur_df_new.groupby('Country_Region', as_index=False)[['Confirmed','Active','Recovered','Deaths']].sum()
#full table showing total confirmed,active,recovered,and deadly cases in each european country
total_overall_by_country

In [ ]:
total_overall_by_country = total_overall_by_country.rename(columns={'Country_Region':'Country'})
#Renaming for Understanding purposes

In [ ]:
total_overall_by_country = total_overall_by_country.set_index('Country')
#set index for y values

In [ ]:
def plotly_dict(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}
#creates a function in order to get the dataframe in the necessary dictionary format for a heatmap, 
#as well as specify the necessary arguments for plotly
         

In [ ]:
fig = go.Figure(data=go.Heatmap(plotly_dict(total_overall_by_country)))
fig.show()
#heatmap of every country's covid cases and labeled. This is a start, want to potentially get 
#geographic JSON data in there to create an actual map

## Thank you for taking a look, please leave feedback!

I am a beginner in using interactive data visualization libraries like plotly and would definitely like some feedback on my code and the analysis as a whole.